- API Response
- 

In [169]:
import requests
import pandas as pd
from datetime import datetime
import pytz

def fetch_tickets(api_url, queryid, cert_path):
    response = requests.get(f"{api_url}{queryid}", cert=cert_path)
    response.raise_for_status()
    return response.json()

def fetch_tickets_with_file(file_path):
    return pd.read_json(file_path, dtype={'ticket_id': 'str'})

def convert_gmt_to_pst(gmt_time_str):
    gmt_time = datetime.strptime(gmt_time_str, "%Y-%m-%dT%H:%M:%S.%fZ")
    gmt_time = pytz.utc.localize(gmt_time)
    pst_time = gmt_time.astimezone(pytz.timezone('US/Pacific'))
    return pst_time.strftime("%Y-%m-%d %H:%M:%S")

def load_tickets_to_table(tickets):
    ticket_data = []
    for ticket in tickets:
        ticket_data.append({
            "ticket_id": ticket["ticket_id"],
            "createdAt": convert_gmt_to_pst(ticket["createdAt"]),
            "title": ticket["title"],
            "status": ticket["status"],
            "priority": ticket["priority"],
            "assignee": ticket["assignee"]["name"],
            "component_name": ticket["component"]["name"],
            "component_version": ticket["component"]["version"],
            "lastModifiedAt": convert_gmt_to_pst(ticket["lastModifiedAt"]),
            "resolvedAt": convert_gmt_to_pst(ticket.get("resolvedAt", "N/A")),
            "resolvedBy": ticket["resolvedBy"]["name"],
            "originator": ticket["originator"]["name"],
            "description": ticket["description"],
            "diagnosis": ticket["diagnosis"]
        })
    return pd.DataFrame(ticket_data)

# Example usage
api_url = "https://sampleapiurl.xyz.com/services/tickets/"
queryid = "123456"
cert_path = "path/to/internal_certificate.pem"
# tickets = fetch_tickets(api_url, queryid, cert_path)
# ticket_df = load_tickets_to_table(tickets)
ticket_df = fetch_tickets_with_file('/Users/koladiya/api_responses.json')
# ticket_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ticket_id       10 non-null     int64 
 1   createdAt       10 non-null     object
 2   title           10 non-null     object
 3   status          10 non-null     object
 4   priority        10 non-null     object
 5   assignee        10 non-null     object
 6   component       10 non-null     object
 7   lastModifiedAt  10 non-null     object
 8   resolvedAt      10 non-null     object
 9   resolvedBy      10 non-null     object
 10  originator      10 non-null     object
 11  description     10 non-null     object
 12  diagnosis       10 non-null     object
dtypes: int64(1), object(12)
memory usage: 1.1+ KB


In [176]:
# def load_exclude_patterns():
#     return [
#         "Misalignment",
#     ]

def load_file_as_list(file_path):
    # with open(file_path, 'r') as file:
    #     return list(line.strip() for line in file)
    with open(file_path, 'r') as file:
        return list(line.strip() for line in file)

def load_exclude_file_with_components(file_path):
    return pd.read_json(file_path, dtype={'version': 'str'})

def add_exclude_ind(ticket_df, description_patterns_file, ticket_ids_file, component_file):
    description_patterns = load_file_as_list(description_patterns_file)
    excluded_ids = load_file_as_list(ticket_ids_file)
    # print("excluded_ids are {}, type: {}, type: {}".format(excluded_ids, type(excluded_ids), type(excluded_ids[0])))
    excluded_components = load_exclude_file_with_components(component_file)
    print(excluded_components)
    
    def match_patterns(description):
        for pattern in description_patterns:
            if(pattern in description):
               return True
        return False
        # return any(pattern in description for pattern in patterns)

    def match_id(id):
        for excluded_id in excluded_ids:
            if(id == excluded_id):
               return True
        return False

    def match_component(component_name, component_version):
        # print("name: {}, version: {}, excluded_components: {}".format(component_name, component_version, excluded_components))
        # print(type(excluded_components['name'].values.tolist()))
        name_list = excluded_components['name'].values.tolist()
        version_list = excluded_components['version'].values.tolist()
        # print(excluded_components['version'].info())
        if(component_name in name_list or component_version in version_list):
            return True
        else:
            return False

    def exclusion_lambda(row): 
        derived_row = pd.json_normalize(row['component'])
        if match_id(row['ticket_id']):
            return 'Y'
        elif match_patterns(row['description']):
            return 'Y'
        elif match_component(derived_row.loc[0, 'name'], derived_row.loc[0, 'version']):
            return 'Y'
        else:
            return 'N'


    ticket_df['exclude_ind'] = ticket_df.apply(lambda row: exclusion_lambda(row), axis=1)
    
    # ticket_df['exclude_ind'] = 'N'
    # for idx, row in ticket_df.iterrows():
    #     derived_row = pd.json_normalize(row['component'])
    #     # print(derived_row.loc[0, 'name'])
    #     # print(derived_row.loc[0, 'version'])
    #     # print("ticket_id: {}, type: {}".format(row['ticket_id'], type(row['ticket_id'])))

    #     # if row['ticket_id'] in excluded_ids:
    #     if match_id(row['ticket_id']):
    #         # print("coming_here ticket: {}".format(row))
    #         ticket_df.at[idx, 'exclude_ind'] = 'Y'
    #     elif match_patterns(row['description']):
    #         ticket_df.at[idx, 'exclude_ind'] = 'Y'
    #     elif match_component(derived_row.loc[0, 'name'], derived_row.loc[0, 'version']):
    #         ticket_df.at[idx, 'exclude_ind'] = 'Y'

# Example usage
description_patterns_file = "/Users/koladiya/patterns.txt"
ticket_ids_file = "/Users/koladiya/exclude_ids.txt"
component_file = "/Users/koladiya/exclude_components.txt"
add_exclude_ind(ticket_df, description_patterns_file, ticket_ids_file, component_file)
ticket_df

          name version
0   Component9     6.0
1  Component11     7.0


,ticket_id,createdAt,title,status,priority,assignee,component,lastModifiedAt,resolvedAt,resolvedBy,originator,description,diagnosis,exclude_ind,issue_type
0,1,2024-08-24T08:00:00.000Z,Issue with display,Open,High,"{'id': 'a1', 'email': 'teamA@example.com', 'na...","{'name': 'Component1', 'version': '1.0'}",2024-08-24T09:00:00.000Z,2024-08-24T12:00:00.000Z,"{'id': 'b1', 'email': 'resolver@example.com', ...","{'id': 'c1', 'email': 'originator@example.com'...",signal Issue with display alignment on the mai...,signal The values display data module has a is...,N,Data
1,2,2024-08-24T09:00:00.000Z,API authentication failure,Closed,Low,"{'id': 'b2', 'email': 'teamB@example.com', 'na...","{'name': 'Component2', 'version': '2.0'}",2024-08-24T10:00:00.000Z,N/A,"{'id': 'd1', 'email': 'resolverB@example.com',...","{'id': 'e1', 'email': 'originatorB@example.com...",Failed authentication with API endpoint.,Authentication token expired or invalid.,Y,API
2,3,2024-08-24T10:00:00.000Z,Data load failure,Open,Medium,"{'id': 'a1', 'email': 'teamA@example.com', 'na...","{'name': 'Component1', 'version': '1.0'}",2024-08-24T11:00:00.000Z,N/A,"{'id': 'c2', 'email': 'resolverC@example.com',...","{'id': 'd2', 'email': 'originatorC@example.com...",Failure in loading data from source X.,Possible data source connectivity issue.,Y,Data
3,4,2024-08-24T11:00:00.000Z,Display misalignment,Closed,High,"{'id': 'b3', 'email': 'teamC@example.com', 'na...","{'name': 'Component3', 'version': '3.0'}",2024-08-24T12:00:00.000Z,2024-08-24T13:00:00.000Z,"{'id': 'e2', 'email': 'resolverD@example.com',...","{'id': 'f1', 'email': 'originatorD@example.com...",Misalignment on the display component.,Component layout adjustment required.,N,Display
4,5,2024-08-24T11:00:00.000Z,Display misalignment 1,Closed,High,"{'id': 'b3', 'email': 'teamC@example.com', 'na...","{'name': 'Component5', 'version': '5.0'}",2024-08-24T12:00:00.000Z,2024-08-24T13:00:00.000Z,"{'id': 'e2', 'email': 'resolverD@example.com',...","{'id': 'f1', 'email': 'originatorD@example.com...",Misalignment5 on the display component.,Component layout adjustment required.,N,Display
5,6,2024-08-24T11:00:00.000Z,Display misalignment,Closed,High,"{'id': 'b3', 'email': 'teamC@example.com', 'na...","{'name': 'Component6', 'version': '6.0'}",2024-08-24T12:00:00.000Z,2024-08-24T13:00:00.000Z,"{'id': 'e2', 'email': 'resolverD@example.com',...","{'id': 'f1', 'email': 'originatorD@example.com...",Misalignment6 on the display component.,Component layout adjustment required.,Y,Display
6,7,2024-08-24T11:00:00.000Z,Display misalignment,Closed,High,"{'id': 'b3', 'email': 'teamC@example.com', 'na...","{'name': 'Component7', 'version': '7.0'}",2024-08-24T12:00:00.000Z,2024-08-24T13:00:00.000Z,"{'id': 'e2', 'email': 'resolverD@example.com',...","{'id': 'f1', 'email': 'originatorD@example.com...",Misalignment7 on the display component.,Component layout adjustment required.,Y,Display
7,8,2024-08-24T11:00:00.000Z,Display misalignment,Closed,High,"{'id': 'b3', 'email': 'teamC@example.com', 'na...","{'name': 'Component8', 'version': '8.0'}",2024-08-24T12:00:00.000Z,2024-08-24T13:00:00.000Z,"{'id': 'e2', 'email': 'resolverD@example.com',...","{'id': 'f1', 'email': 'originatorD@example.com...",Misalignment8 on the display component.,Component layout adjustment required.,N,Display
8,9,2024-08-24T11:00:00.000Z,Display misalignment,Closed,High,"{'id': 'b3', 'email': 'teamC@example.com', 'na...","{'name': 'Component9', 'version': '3.0'}",2024-08-24T12:00:00.000Z,2024-08-24T13:00:00.000Z,"{'id': 'e2', 'email': 'resolverD@example.com',...","{'id': 'f1', 'email': 'originatorD@example.com...",Misalignment9 on the display component.,Component layout adjustment required.,Y,Display
9,10,2024-08-24T11:00:00.000Z,Display misalignment,Closed,High,"{'id': 'b3', 'email': 'teamC@example.com', 'na...","{'name': 'Component10', 'version': '10.0'}",2024-08-24T12:00:00.000Z,2024-08-24T13:00:00.000Z,"{'id': 'e2', 'email': 'resolverD@example.com',...","{'id': 'f1', 'em

In [185]:
def load_issue_type_keywords():
    return {
        "Display": ["landscape", "image", "artwork", "misalignment"],
        "Infrastructure": ["pipeline", "Infra", "structure", "platform"],
        "Data": ["data", "data load", "values", "signal"],
        "API": ["api", "auth", "interface", "usage"]
    }

# def load_issue_type(row):
#     for issue_type, words in keywords.items():
#         for field in text_fields:



def determine_issue_type(row, keywords):
    # text_fields = [row['title'], row['description'], row['diagnosis']]
    title = row['title']
    description = row['description']
    diagnosis = row['diagnosis']

    # for issue_type in keywords:
            
    # type_scores = {issue_type: 0 for issue_type in keywords}
    # print(type(type_scores))
    type_scores = {}
    for issue_type in keywords:
        type_scores[issue_type] = 0

    # print(type_scores)
    
    for issue_type, words in keywords.items():
        for word in words:
            type_scores[issue_type] += title.lower().count(word.lower())
            type_scores[issue_type] += description.lower().count(word.lower())
            type_scores[issue_type] += diagnosis.lower().count(word.lower())
        # for field in text_fields:
            # for word in words:
            #     type_scores[issue_type] += field.lower().count(word.lower())

    print(type_scores)
    max_freq = -1
    answer = ""
    for issue_type, freq in type_scores.items():
        if freq == max_freq:
            None
            # tie breaker
        if freq > max_freq:
            max_freq = freq
            answer = issue_type

    return answer  
    # return max(type_scores, key=type_scores.get)

def add_issue_type(ticket_df):
    keywords = load_issue_type_keywords()
    ticket_df['issue_type'] = ticket_df.apply(lambda row: determine_issue_type(row, keywords), axis=1)

# Example usage
add_issue_type(ticket_df)
ticket_df

{'Display': 0, 'Infrastructure': 0, 'Data': 4, 'API': 0}
{'Display': 0, 'Infrastructure': 0, 'Data': 0, 'API': 5}
{'Display': 0, 'Infrastructure': 0, 'Data': 4, 'API': 0}
{'Display': 2, 'Infrastructure': 0, 'Data': 0, 'API': 0}
{'Display': 2, 'Infrastructure': 0, 'Data': 0, 'API': 0}
{'Display': 2, 'Infrastructure': 0, 'Data': 0, 'API': 0}
{'Display': 2, 'Infrastructure': 0, 'Data': 0, 'API': 0}
{'Display': 2, 'Infrastructure': 0, 'Data': 0, 'API': 0}
{'Display': 2, 'Infrastructure': 0, 'Data': 0, 'API': 0}
{'Display': 2, 'Infrastructure': 0, 'Data': 0, 'API': 0}


,ticket_id,createdAt,title,status,priority,assignee,component,lastModifiedAt,resolvedAt,resolvedBy,originator,description,diagnosis,exclude_ind,issue_type
0,1,2024-08-24T08:00:00.000Z,Issue with display,Open,High,"{'id': 'a1', 'email': 'teamA@example.com', 'na...","{'name': 'Component1', 'version': '1.0'}",2024-08-24T09:00:00.000Z,2024-08-24T12:00:00.000Z,"{'id': 'b1', 'email': 'resolver@example.com', ...","{'id': 'c1', 'email': 'originator@example.com'...",signal Issue with display alignment on the mai...,signal The values display data module has a is...,N,Data
1,2,2024-08-24T09:00:00.000Z,API authentication failure,Closed,Low,"{'id': 'b2', 'email': 'teamB@example.com', 'na...","{'name': 'Component2', 'version': '2.0'}",2024-08-24T10:00:00.000Z,N/A,"{'id': 'd1', 'email': 'resolverB@example.com',...","{'id': 'e1', 'email': 'originatorB@example.com...",Failed authentication with API endpoint.,Authentication token expired or invalid.,Y,API
2,3,2024-08-24T10:00:00.000Z,Data load failure,Open,Medium,"{'id': 'a1', 'email': 'teamA@example.com', 'na...","{'name': 'Component1', 'version': '1.0'}",2024-08-24T11:00:00.000Z,N/A,"{'id': 'c2', 'email': 'resolverC@example.com',...","{'id': 'd2', 'email': 'originatorC@example.com...",Failure in loading data from source X.,Possible data source connectivity issue.,Y,Data
3,4,2024-08-24T11:00:00.000Z,Display misalignment,Closed,High,"{'id': 'b3', 'email': 'teamC@example.com', 'na...","{'name': 'Component3', 'version': '3.0'}",2024-08-24T12:00:00.000Z,2024-08-24T13:00:00.000Z,"{'id': 'e2', 'email': 'resolverD@example.com',...","{'id': 'f1', 'email': 'originatorD@example.com...",Misalignment on the display component.,Component layout adjustment required.,N,Display
4,5,2024-08-24T11:00:00.000Z,Display misalignment 1,Closed,High,"{'id': 'b3', 'email': 'teamC@example.com', 'na...","{'name': 'Component5', 'version': '5.0'}",2024-08-24T12:00:00.000Z,2024-08-24T13:00:00.000Z,"{'id': 'e2', 'email': 'resolverD@example.com',...","{'id': 'f1', 'email': 'originatorD@example.com...",Misalignment5 on the display component.,Component layout adjustment required.,N,Display
5,6,2024-08-24T11:00:00.000Z,Display misalignment,Closed,High,"{'id': 'b3', 'email': 'teamC@example.com', 'na...","{'name': 'Component6', 'version': '6.0'}",2024-08-24T12:00:00.000Z,2024-08-24T13:00:00.000Z,"{'id': 'e2', 'email': 'resolverD@example.com',...","{'id': 'f1', 'email': 'originatorD@example.com...",Misalignment6 on the display component.,Component layout adjustment required.,Y,Display
6,7,2024-08-24T11:00:00.000Z,Display misalignment,Closed,High,"{'id': 'b3', 'email': 'teamC@example.com', 'na...","{'name': 'Component7', 'version': '7.0'}",2024-08-24T12:00:00.000Z,2024-08-24T13:00:00.000Z,"{'id': 'e2', 'email': 'resolverD@example.com',...","{'id': 'f1', 'email': 'originatorD@example.com...",Misalignment7 on the display component.,Component layout adjustment required.,Y,Display
7,8,2024-08-24T11:00:00.000Z,Display misalignment,Closed,High,"{'id': 'b3', 'email': 'teamC@example.com', 'na...","{'name': 'Component8', 'version': '8.0'}",2024-08-24T12:00:00.000Z,2024-08-24T13:00:00.000Z,"{'id': 'e2', 'email': 'resolverD@example.com',...","{'id': 'f1', 'email': 'originatorD@example.com...",Misalignment8 on the display component.,Component layout adjustment required.,N,Display
8,9,2024-08-24T11:00:00.000Z,Display misalignment,Closed,High,"{'id': 'b3', 'email': 'teamC@example.com', 'na...","{'name': 'Component9', 'version': '3.0'}",2024-08-24T12:00:00.000Z,2024-08-24T13:00:00.000Z,"{'id': 'e2', 'email': 'resolverD@example.com',...","{'id': 'f1', 'email': 'originatorD@example.com...",Misalignment9 on the display component.,Component layout adjustment required.,Y,Display
9,10,2024-08-24T11:00:00.000Z,Display misalignment,Closed,High,"{'id': 'b3', 'email': 'teamC@example.com', 'na...","{'name': 'Component10', 'version': '10.0'}",2024-08-24T12:00:00.000Z,2024-08-24T13:00:00.000Z,"{'id': 'e2', 'email': 'resolverD@example.com',...","{'id': 'f1', 'em